In [511]:
import bs4 as bs
import pandas as pd
import numpy as np
import urllib.request

In [512]:
source = urllib.request.urlopen('http://quotes.toscrape.com').read()

In [513]:
soup = bs.BeautifulSoup(source, 'html.parser')

In [514]:
soup.title

<title>Quotes to Scrape</title>

In [515]:
data = soup.find_all('div', attrs = {'class':'row','class':'quote'})
#authors = soup.find_all_next('div', class_ = 'quote')
authors = soup.find_all('div', attrs={'class':'quote'})


In [90]:
quotes = []
authors_all = []
articles = []
for i in data:
    text = i.find_all('span', class_='text')
    for j in text:
        quotes.append(j.text.strip())


articles.append({
    'quote':quotes,
    'author':authors_all
})
    
    
type(quotes)

list

In [85]:
quotes = []
authors_all = []
articles = []
for i in data:
    text = i.find('span', class_='text')
    quotes.append(text.text.strip())
    authors = soup.find('div', class_='quote')
   # print(authors)
    author_data = authors.find_all('small', class_='author')
    print(author_data)
 #   for j in author_data:
  #      print(j.text)
#    print(author_data.text)
    
#    authors_span = authors.find('small') 
#    print(authors_span)
   # authors_all.append(authors_span.text)




articles.append({
    'quote':quotes,
    'author':authors_all
})
    
    
#articles

[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]
[<small class="author" itemprop="author">Albert Einstein</small>]


In [99]:
#all_quotes = pd.DataFrame(quotes)
#all_quotes['Authors'] = authors_all
all_quotes

,0,Authors
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


# Creating a DataFrame with Quotes and Authors' names

In [102]:
all_quotes.rename(columns={0:'Quotes'}, inplace=True)

In [103]:
all_quotes

,Quotes,Authors
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


# Getting the list of names of Authors

In [96]:
for i in authors:
    authors_span = i.find('small') if authors else None
    authors_all.append(authors_span.text)
authors_all

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

# Creating a dictionary of Quotes and Authors' names:  articles

In [54]:
quotes = []
authors_all = []
articles = []

for i in data:
    text = i.find_all('span', class_='text')
    for j in text:
        quotes.append(j.text.strip())

for i in authors:
    authors_span = i.find('small') if authors else None
    authors_all.append(authors_span.text)

articles.append({
    'quote':quotes,
    'author':authors_all
})
    

#print(quotes)
#print(authors_all)

In [55]:
articles

[{'quote': ['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
   '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
   '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
   '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
   "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
   '“Try not to become a man of success. Rather become a man of value.”',
   '“It is better to be hated for what you are than to be loved for what you are not.”',
   "“I have not failed. I've just found 10,000 ways that won't work.”",
   "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
   '“A day without sunshine is like, you know, night.”'],
  'author': ['Albert Eins

In [15]:
import requests
import dataset
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

db = dataset.connect('sqlite:///quotes.db')

authors_seen = set()

base_url = 'http://quotes.toscrape.com/'

def clean_url(url):
    # Clean '/author/Steve-Martin' to 'Steve-Martin'
    # Use urljoin to make an absolute URL
    url = urljoin(base_url, url)
    # Use urlparse to get out the path part
    path = urlparse(url).path
    # Now split the path by '/' and get the second part
    # E.g. '/author/Steve-Martin' -> ['', 'author', 'Steve-Martin']
    return path.split('/')[2]

def scrape_quotes(html_soup):
    for quote in html_soup.select('div.quote'):
        quote_text = quote.find(class_='text').get_text(strip=True)
        quote_author_url = clean_url(quote.find(class_='author').find_next_sibling('a').get('href'))
        quote_tag_urls = [clean_url(a.get('href')) for a in quote.find_all('a', class_='tag')]
        authors_seen.add(quote_author_url)
        # Store this quote and its tags
        quote_id = db['quotes'].insert({ 'text' : quote_text,
                                         'author' : quote_author_url })
        db['quote_tags'].insert_many(
                 [{'quote_id' : quote_id, 'tag_id' : tag} for tag in quote_tag_urls])

def scrape_author(html_soup, author_id):
    author_name = html_soup.find(class_='author-title').get_text(strip=True)
    author_born_date = html_soup.find(class_='author-born-date').get_text(strip=True)
    author_born_loc = html_soup.find(class_='author-born-location').get_text(strip=True)
    author_description = html_soup.find(class_='author-description').get_text(strip=True)
    db['authors'].insert({ 'author_id' : author_id,
                           'name' : author_name,
                           'born_date' : author_born_date,
                           'born_location' : author_born_loc,
                           'description' : author_description})

# Start by scraping all the quote pages
url = base_url
while True:
    print('Now scraping page:', url)
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    # Scrape the quotes
    scrape_quotes(html_soup)
    # Is there a next page?
    next_a = html_soup.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url = urljoin(url, next_a[0].get('href'))

# Now fetch out the author information
for author_id in authors_seen:
    url = urljoin(base_url, '/author/' + author_id)
    print('Now scraping author:', url)
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    # Scrape the author information
    scrape_author(html_soup, author_id)

Now scraping page: http://quotes.toscrape.com/
Now scraping page: http://quotes.toscrape.com/page/2/
Now scraping page: http://quotes.toscrape.com/page/3/
Now scraping page: http://quotes.toscrape.com/page/4/
Now scraping page: http://quotes.toscrape.com/page/5/
Now scraping page: http://quotes.toscrape.com/page/6/
Now scraping page: http://quotes.toscrape.com/page/7/
Now scraping page: http://quotes.toscrape.com/page/8/
Now scraping page: http://quotes.toscrape.com/page/9/
Now scraping page: http://quotes.toscrape.com/page/10/
Now scraping author: http://quotes.toscrape.com/author/Charles-Bukowski
Now scraping author: http://quotes.toscrape.com/author/Steve-Martin
Now scraping author: http://quotes.toscrape.com/author/Dr-Seuss
Now scraping author: http://quotes.toscrape.com/author/J-K-Rowling
Now scraping author: http://quotes.toscrape.com/author/Alexandre-Dumas-fils
Now scraping author: http://quotes.toscrape.com/author/Harper-Lee
Now scraping author: http://quotes.toscrape.com/autho

# NEW APPROACH- CONSOLIDATED

In [122]:
data = soup.find_all('div', attrs = {'class':'row','class':'quote'})
#authors = soup.find_all_next('div', class_ = 'quote')
authors = soup.find_all('div', attrs={'class':'quote'})



In [13]:
base_url = 'http://quotes.toscrape.com/'
def clean_url(url):
    url = urljoin(base_url, url)
    path = urlparse(url).path
    return url.split('/')[2]

    
    

# Getting the tags and their URLs

In [115]:
all_tags = []
for i in soup.find_all('div', class_ = 'col-md-4 tags-box'):
    tags = i.find_all('a', href = True)
    for j in tags:
        all_tags.append(j.get('href'))

In [116]:
base_url = 'http://quotes.toscrape.com/'
tag_urls = []
for tag_url in all_tags:
    url_tag = urljoin(base_url, tag_url)
    tag_urls.append(url_tag)


In [162]:
tag_urls

['http://quotes.toscrape.com/tag/love/',
 'http://quotes.toscrape.com/tag/inspirational/',
 'http://quotes.toscrape.com/tag/life/',
 'http://quotes.toscrape.com/tag/humor/',
 'http://quotes.toscrape.com/tag/books/',
 'http://quotes.toscrape.com/tag/reading/',
 'http://quotes.toscrape.com/tag/friendship/',
 'http://quotes.toscrape.com/tag/friends/',
 'http://quotes.toscrape.com/tag/truth/',
 'http://quotes.toscrape.com/tag/simile/']

# Getting the quotes text

In [255]:
for page in tag_urls:
    source = urllib.request.urlopen(page).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})

def quotes1():
    source = urllib.request.urlopen("http://quotes.toscrape.com/tag/love/").read()
    new_list = []
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())

            #print(j.text.strip())

    return len(new_list)
print(quotes1())

10


In [222]:
quotes(data)

7
7
7
7
7
7
7
7
7
7


In [204]:
def author_name(authors):
    authors_all11 = []
    for i in authors:
        authors_span = i.find_all('small')
        author_name = (authors_span.text)
    return author_name

# Main Program

In [205]:
quotes_statement = []
author_name_new = []
for page in tag_urls:
#    print(page)
    source = urllib.request.urlopen(page).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'quote'})
    #authors = soup.find_all('div', attrs={'class':'quote'})
    quote_statement.append(quotes(data))
#quotes_new.append(quotes(data))
#    author_name_new =author_name(authors)



In [208]:
for k in quote_statement:
    print(k.split(','))

['“A day without sunshine is like', ' you know', ' night.”']
['“Life is like riding a bicycle. To keep your balance', ' you must keep moving.”']
["“He's like a drug for you", ' Bella.”']


AttributeError: 'list' object has no attribute 'split'

In [142]:
quotes1

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

In [119]:
quotes = []
authors_all = []
articles = []

for i in data:
    text = i.find_all('span', class_='text')
    for j in text:
        quotes.append(j.text.strip())

for i in authors:
    authors_span = i.find('small') if authors else None
    authors_all.append(authors_span.text)

articles.append({
    'quote':quotes,
    'author':authors_all
})

AttributeError: 'int' object has no attribute 'text'

In [ ]:
quotes = []
authors_all = []
articles = []
for i in data:
    text = i.find_all('span', class_='text')
    for j in text:
        quotes.append(j.text.strip())

In [312]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request
from urllib.parse import urlparse, urljoin


base_url = 'http://quotes.toscrape.com/'
source = urllib.request.urlopen(base_url).read()
soup = bs.BeautifulSoup(source, 'html.parser')


#function to get the quotes text from the webpage
def quotes(url):
    new_list = []
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())

    return new_list

# Getting the URLs
def get_urls(soup):
    all_tags = []
    for i in soup.find_all('div', class_ = 'col-md-4 tags-box'):
        tags = i.find_all('a', href = True)
        for j in tags:
            all_tags.append(j.get('href'))
    base_url = 'http://quotes.toscrape.com/'
    tag_urls = []
    for tag_url in all_tags:
        url_tag = urljoin(base_url, tag_url)
        tag_urls.append(url_tag)
    return tag_urls


#Main function
all_quotes = []
urls = get_urls(soup)
for url1 in urls:
   # print(url1)
    quote_l = quotes(url1)
   # print(quote_l)
    next_page = soup.select('li.next > a')
    if not next_page or not next_page[0].get('href'):
        break
    url1 = url1 + next_page[0].get('href').lstrip('/')
   # print(url1)
    all_quotes.append(quote_l)
    #print(len(all_quotes))

for url2 in urls:
    author
quotes_to_scrap = pd.DataFrame()
quotes_to_scrap['Quotes'] = new_list









In [313]:
quotes_to_scrap

,Quotes
0,“It is better to be hated for what you are tha...
1,“This life is what you make it. No matter what...
2,"“You may not be her first, her last, or her on..."
3,"“The opposite of love is not hate, it's indiff..."
4,"“It is not a lack of love, but a lack of frien..."
...,...
68,"“The truth."" Dumbledore sighed. ""It is a beaut..."
69,“Never tell the truth to people who are not wo...
70,"“A day without sunshine is like, you know, nig..."
71,“Life is like riding a bicycle. To keep your b...


In [310]:
new_list = []
for i in all_quotes:
    for j in i:
        new_list.append(j)
len(new_list

73

In [289]:

string = "http://quotes.toscrape.com/tag/humor/"+"page/2"
string

'http://quotes.toscrape.com/tag/humor/page/2'

# Get authors links

In [337]:
def get_author_links(url):
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    author_links = []
    for i in data:    
        author_link = i.select('a')
        for j in author_link:
            author_link2 = j.get('href')
            if 'author' in author_link2:
                author_links.append(author_link2)
            
    return author_links




    

# Get author's info

In [492]:
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source, 'html.parser')

urls = get_urls(soup)
#all_author_urls = []
for url in urls:
    print(url)
    #next_page = soup.select('li.next > a')
    #if not next_page or not next_page[0].get('href'):
     #   break
    #url = url + next_page[0].get('href').lstrip('/')
#    print(url)
#    author_urls = get_author_links(url)
#    print(author_urls)
    
    
    
    
 #   for j in author_urls:
  #      final_url = base_url + j.lstrip('/')
   #     if final_url not in all_author_urls:
    #        all_author_urls.append(final_url)

#all_author_urls



http://quotes.toscrape.com/tag/love/
http://quotes.toscrape.com/tag/inspirational/
http://quotes.toscrape.com/tag/life/
http://quotes.toscrape.com/tag/humor/
http://quotes.toscrape.com/tag/books/
http://quotes.toscrape.com/tag/reading/
http://quotes.toscrape.com/tag/friendship/
http://quotes.toscrape.com/tag/friends/
http://quotes.toscrape.com/tag/truth/
http://quotes.toscrape.com/tag/simile/


In [334]:
base_url

'http://quotes.toscrape.com/'

# Getting Authors info URLs


In [504]:
source = urllib.request.urlopen(base_url).read()
soup = bs.BeautifulSoup(source, 'html.parser')

urls = get_urls(soup)
all_author_urls = []
for url in urls:
#    print(url)
    author_urls = get_author_links(url)
   # print(author_urls)
    for j in author_urls:
        #print(j)
        final_url = base_url + j.lstrip('/')
        all_author_urls.append(final_url)
        #print(final_url)
   #     if final_url not in all_author_urls:
    #

all_author_urls

['http://quotes.toscrape.com/author/Andre-Gide',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Bob-Marley',
 'http://quotes.toscrape.com/author/Elie-Wiesel',
 'http://quotes.toscrape.com/author/Friedrich-Nietzsche',
 'http://quotes.toscrape.com/author/Pablo-Neruda',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/James-Baldwin',
 'http://quotes.toscrape.com/author/Jane-Austen',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Thomas-A-Edison',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Elie-Wiesel',
 'http://quotes.toscrape.com/author/J-K-Rowling',
 'http://quotes.toscrape.com/author/George-Eliot',
 'http://quotes.toscrape.com/author/C-S-Lewis',
 'http://quotes.toscrape.com/author/Martin-Luther-King-Jr',
 'http://

# Getting Birthdates and Birthplaces

In [527]:
#print(type(all_author_urls))

def get_author_birthdate(all_author_urls):
    author_birthdates = []
    author_birthplaces = []
    author_info = {}
    for author_url_single in all_author_urls:
        source_author = urllib.request.urlopen(author_url_single).read()
        soup = bs.BeautifulSoup(source_author, 'html.parser')
        data_author = soup.select('p')
        for author in data_author:
            birthdate = author.find_all('span',class_='author-born-date')
            for i in birthdate:
                author_birthdates.append(i.text)
        for author in data_author:
            birthplace = author.find_all('span', class_='author-born-location')
            for j in birthplace:
                author_birthplaces.append(j.text)
         
                author_info = {
                'birthdate':author_birthdates,
                'birthplace':author_birthplaces
                }
    return author_birthdates, author_birthplaces





In [510]:
birthdates

['November 22, 1869',
 'June 01, 1926',
 'February 06, 1945',
 'September 30, 1928',
 'October 15, 1844',
 'July 12, 1904',
 'June 01, 1926',
 'June 01, 1926',
 'August 02, 1924',
 'December 16, 1775',
 'March 14, 1879',
 'June 01, 1926',
 'February 11, 1847',
 'June 01, 1926',
 'September 30, 1928',
 'July 31, 1965',
 'November 22, 1819',
 'November 29, 1898',
 'January 15, 1929',
 'June 27, 1880',
 'March 14, 1879',
 'November 22, 1869',
 'June 01, 1926',
 'March 11, 1952',
 'November 30, 1835',
 'April 24, 1899',
 'March 02, 1904',
 'March 14, 1879',
 'July 26, 1856',
 'May 25, 1803',
 'December 16, 1775',
 'August 14, 1945',
 'August 07, 1942',
 'September 24, 1936',
 'November 26, 1922',
 'August 11, 1962',
 'August 16, 1920',
 'April 28, 1948',
 'March 02, 1904',
 'May 12, 1937',
 'December 16, 1775',
 'November 30, 1835',
 'August 24, 1899',
 'November 29, 1898',
 'January 12, 1949',
 'July 21, 1899',
 'January 01, 1919',
 'November 30, 1835',
 'December 16, 1775',
 'November 29

In [509]:
birthdates,birthplaces = get_author_birthdate(all_author_urls)

In [415]:
quotes_to_scrap['a']

,Quotes
0,“It is better to be hated for what you are tha...
1,“This life is what you make it. No matter what...
2,"“You may not be her first, her last, or her on..."
3,"“The opposite of love is not hate, it's indiff..."
4,"“It is not a lack of love, but a lack of frien..."
...,...
68,"“The truth."" Dumbledore sighed. ""It is a beaut..."
69,“Never tell the truth to people who are not wo...
70,"“A day without sunshine is like, you know, nig..."
71,“Life is like riding a bicycle. To keep your b...


In [477]:
def get_author_links(url):
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    author_links = []
    for i in data:
        author_link = i.select('a')
        for j in author_link:
            author_link2 = j.get('href')
            if 'author' in author_link2:
                author_links.append(author_link2)

    return author_links
get_author_links(url)

['/author/Steve-Martin', '/author/Albert-Einstein', '/author/Stephenie-Meyer']

In [450]:
def quotes(url):
    new_list = []
    author_names = []
    final_list = []
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())
    for author in data:
        author_name = author.find_all('small', class_='author')
        for a_n in author_name:
            author_names.append(a_n.text.strip())
    
        final_list.append({
            'quotes':new_list,
            'authors':author_names
        })
    return author_names, new_list


urls = get_urls(soup)
for url1 in urls:
    authors_l,quote1 = quotes(url1)
#    for i in authors:
 #       print(i)
    print(authors_l)

10
10
10
10
10
7
5
4
4
3


In [424]:

   # quote_l = quotes(url1)

def quotes(url):
    new_list = []
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())

    return new_list
all_quotes = []
urls = get_urls(soup)
for url1 in urls:
    quote = quotes(url1)
    print(quote)

['“It is better to be hated for what you are than to be loved for what you are not.”', "“This life is what you make it. No matter what, you're going to mess up sometimes, it's a universal truth. But the good part is you get to decide how you're going to mess it up. Girls will be your friends - they'll act like it anyway. But just remember, some come, some go. The ones that stay with you through everything - they're your true best friends. Don't let go of them. Also remember, sisters make the best friends in the world. As for lovers, well, they'll come and go too. And baby, I hate to say it, most of them - actually pretty much all of them are going to break your heart, but you can't give up because if you give up, you'll never find your soulmate. You'll never find that half who makes you whole and that goes for everything. Just because you fail once, doesn't mean you're gonna fail at everything. Keep trying, hold on, and always, always, always believe in yourself, because if you don't, 

["“The more that you read, the more things you will know. The more that you learn, the more places you'll go.”", "“We read to know we're not alone.”", '“A reader lives a thousand lives before he dies, said Jojen. The man who never reads lives only one.”', '“You can never get a cup of tea large enough or a book long enough to suit me.”', "“What really knocks me out is a book that, when you're all done reading it, you wish the author that wrote it was a terrific friend of yours and you could call him up on the phone whenever you felt like it. That doesn't happen much, though.”", "“′Classic′ - a book which people praise and don't read.”", '“I declare after all there is no enjoyment like reading! How much sooner one tires of any thing than of a book! -- When I have a house of my own, I shall be miserable if I have not an excellent library.”']
['“It is not a lack of love, but a lack of friendship that makes unhappy marriages.”', '“Good friends, good books, and a sleepy conscience: this is t

In [422]:
quotes(url)

['“A day without sunshine is like, you know, night.”',
 '“Life is like riding a bicycle. To keep your balance, you must keep moving.”',
 "“He's like a drug for you, Bella.”"]

In [439]:
quotes_to_scrap['author'] = all_authors
quotes_to_scrap

ValueError: Length of values does not match length of index

In [446]:
all_authors = []
for author in authors_l:
    print(author)
#    all_authors.append(author)
#len(all_authors)

Steve Martin
Albert Einstein
Stephenie Meyer


In [453]:
urls = get_urls(soup)
all_authors3 = []
for url in urls:
    author3, quotes3 = quotes(url)
    all_authors3.append(author3)
    
print(all_authors3)

[['André Gide', 'Marilyn Monroe', 'Bob Marley', 'Elie Wiesel', 'Friedrich Nietzsche', 'Pablo Neruda', 'Marilyn Monroe', 'Marilyn Monroe', 'James Baldwin', 'Jane Austen'], ['Albert Einstein', 'Marilyn Monroe', 'Thomas A. Edison', 'Marilyn Monroe', 'Elie Wiesel', 'J.K. Rowling', 'George Eliot', 'C.S. Lewis', 'Martin Luther King Jr.', 'Helen Keller'], ['Albert Einstein', 'André Gide', 'Marilyn Monroe', 'Douglas Adams', 'Mark Twain', 'Allen Saunders', 'Dr. Seuss', 'Albert Einstein', 'George Bernard Shaw', 'Ralph Waldo Emerson'], ['Jane Austen', 'Steve Martin', 'Garrison Keillor', 'Jim Henson', 'Charles M. Schulz', 'Suzanne Collins', 'Charles Bukowski', 'Terry Pratchett', 'Dr. Seuss', 'George Carlin'], ['Jane Austen', 'Mark Twain', 'Jorge Luis Borges', 'C.S. Lewis', 'Haruki Murakami', 'Ernest Hemingway', 'J.D. Salinger', 'Mark Twain', 'Jane Austen', "Madeleine L'Engle"], ['Dr. Seuss', 'William Nicholson', 'George R.R. Martin', 'C.S. Lewis', 'J.D. Salinger', 'Mark Twain', 'Jane Austen'], ['F

In [456]:
final_authors = []
for authors in all_authors3:
    for author_name in authors:
        final_authors.append(author_name)
print(final_authors)


['André Gide', 'Marilyn Monroe', 'Bob Marley', 'Elie Wiesel', 'Friedrich Nietzsche', 'Pablo Neruda', 'Marilyn Monroe', 'Marilyn Monroe', 'James Baldwin', 'Jane Austen', 'Albert Einstein', 'Marilyn Monroe', 'Thomas A. Edison', 'Marilyn Monroe', 'Elie Wiesel', 'J.K. Rowling', 'George Eliot', 'C.S. Lewis', 'Martin Luther King Jr.', 'Helen Keller', 'Albert Einstein', 'André Gide', 'Marilyn Monroe', 'Douglas Adams', 'Mark Twain', 'Allen Saunders', 'Dr. Seuss', 'Albert Einstein', 'George Bernard Shaw', 'Ralph Waldo Emerson', 'Jane Austen', 'Steve Martin', 'Garrison Keillor', 'Jim Henson', 'Charles M. Schulz', 'Suzanne Collins', 'Charles Bukowski', 'Terry Pratchett', 'Dr. Seuss', 'George Carlin', 'Jane Austen', 'Mark Twain', 'Jorge Luis Borges', 'C.S. Lewis', 'Haruki Murakami', 'Ernest Hemingway', 'J.D. Salinger', 'Mark Twain', 'Jane Austen', "Madeleine L'Engle", 'Dr. Seuss', 'William Nicholson', 'George R.R. Martin', 'C.S. Lewis', 'J.D. Salinger', 'Mark Twain', 'Jane Austen', 'Friedrich Niet

In [457]:
quotes_to_scrap['Authors'] = final_authors

In [458]:
quotes_to_scrap

,Quotes,Authors
0,“It is better to be hated for what you are tha...,André Gide
1,“This life is what you make it. No matter what...,Marilyn Monroe
2,"“You may not be her first, her last, or her on...",Bob Marley
3,"“The opposite of love is not hate, it's indiff...",Elie Wiesel
4,"“It is not a lack of love, but a lack of frien...",Friedrich Nietzsche
...,...,...
68,"“The truth."" Dumbledore sighed. ""It is a beaut...",J.K. Rowling
69,“Never tell the truth to people who are not wo...,Mark Twain
70,"“A day without sunshine is like, you know, nig...",Steve Martin
71,“Life is like riding a bicycle. To keep your b...,Albert Einstein


In [474]:
def quotes(url):
    new_list = []
    author_names = []
    final_list = []
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())
    for author in data:
        author_name = author.find_all('small', class_='author')
        for a_n in author_name:
            author_names.append(a_n.text.strip())
    
       # final_list.append({
        #    'quotes':new_list,
         #   'authors':author_names
        #})
    return author_names, new_list

urls = get_urls(soup)
all_authors3 = []
final_authors =[]
for url in urls:
    author3, quotes3 = quotes(url)
    for author in author3:
        final_authors.append(author)
        
final_authors

['André Gide',
 'Marilyn Monroe',
 'Bob Marley',
 'Elie Wiesel',
 'Friedrich Nietzsche',
 'Pablo Neruda',
 'Marilyn Monroe',
 'Marilyn Monroe',
 'James Baldwin',
 'Jane Austen',
 'Albert Einstein',
 'Marilyn Monroe',
 'Thomas A. Edison',
 'Marilyn Monroe',
 'Elie Wiesel',
 'J.K. Rowling',
 'George Eliot',
 'C.S. Lewis',
 'Martin Luther King Jr.',
 'Helen Keller',
 'Albert Einstein',
 'André Gide',
 'Marilyn Monroe',
 'Douglas Adams',
 'Mark Twain',
 'Allen Saunders',
 'Dr. Seuss',
 'Albert Einstein',
 'George Bernard Shaw',
 'Ralph Waldo Emerson',
 'Jane Austen',
 'Steve Martin',
 'Garrison Keillor',
 'Jim Henson',
 'Charles M. Schulz',
 'Suzanne Collins',
 'Charles Bukowski',
 'Terry Pratchett',
 'Dr. Seuss',
 'George Carlin',
 'Jane Austen',
 'Mark Twain',
 'Jorge Luis Borges',
 'C.S. Lewis',
 'Haruki Murakami',
 'Ernest Hemingway',
 'J.D. Salinger',
 'Mark Twain',
 'Jane Austen',
 "Madeleine L'Engle",
 'Dr. Seuss',
 'William Nicholson',
 'George R.R. Martin',
 'C.S. Lewis',
 'J.D. S

In [475]:
len(final_authors)

73

In [505]:
author_birthdate, author_birthplace = get_author_birthdate(all_author_urls)

TypeError: get_author_birthdate() takes 0 positional arguments but 1 was given

In [529]:
#for j in all_author_urls:
author_birthdate, author_birthplace = get_author_birthdate(all_author_urls)
   # print(j)
#    final_url = base_url + j.lstrip('/')
 #   all_author_urls.append(j)



HTTPError: HTTP Error 404: NOT FOUND

In [531]:
all_author_urls

['http://quotes.toscrape.com/author/Andre-Gide',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Bob-Marley',
 'http://quotes.toscrape.com/author/Elie-Wiesel',
 'http://quotes.toscrape.com/author/Friedrich-Nietzsche',
 'http://quotes.toscrape.com/author/Pablo-Neruda',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/James-Baldwin',
 'http://quotes.toscrape.com/author/Jane-Austen',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Thomas-A-Edison',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Elie-Wiesel',
 'http://quotes.toscrape.com/author/J-K-Rowling',
 'http://quotes.toscrape.com/author/George-Eliot',
 'http://quotes.toscrape.com/author/C-S-Lewis',
 'http://quotes.toscrape.com/author/Martin-Luther-King-Jr',
 'http://

In [532]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request
from urllib.parse import urlparse, urljoin

# Declaring the base urls - Source of Data
base_url = 'http://quotes.toscrape.com/'
source = urllib.request.urlopen(base_url).read()
soup = bs.BeautifulSoup(source, 'html.parser')


#function to get the quotes text from the webpages
def quotes(url):
    new_list = []
    author_names = []
    final_list = []
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    for i in data:
        text = i.find_all('span', class_='text')
        for j in text:
            new_list.append(j.text.strip())
    for author in data:
        author_name = author.find_all('small', class_='author')
        for a_n in author_name:
            author_names.append(a_n.text.strip())

       # final_list.append({
        #    'quotes':new_list,
         #   'authors':author_names
        #})
    return author_names, new_list

# Getting the URLs
def get_urls(soup):
    all_tags = []
    for i in soup.find_all('div', class_ = 'col-md-4 tags-box'):
        tags = i.find_all('a', href = True)
        for j in tags:
            all_tags.append(j.get('href'))
    base_url = 'http://quotes.toscrape.com/'
    tag_urls = []
    for tag_url in all_tags:
        url_tag = urljoin(base_url, tag_url)
        tag_urls.append(url_tag)
    return tag_urls

# Get author links to get author's personal info
def get_author_links(url):
    source = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'html.parser')
    data = soup.find_all('div', attrs = {'class':'row','class':'col-md-8'})
    author_links = []
    for i in data:
        author_link = i.select('a')
        for j in author_link:
            author_link2 = j.get('href')
            if 'author' in author_link2:
                author_links.append(author_link2)

    return author_links

def get_author_birthdate(all_author_urls):
    author_birthdates = []
    author_birthplaces = []
    author_info = {}
    for author_url_single in all_author_urls:
        source_author = urllib.request.urlopen(author_url_single).read()
        soup = bs.BeautifulSoup(source_author, 'html.parser')
        data_author = soup.select('p')
        for author in data_author:
            birthdate = author.find_all('span',class_='author-born-date')
            for i in birthdate:
                author_birthdates.append(i.text)
        for author in data_author:
            birthplace = author.find_all('span', class_='author-born-location')
            for j in birthplace:
                author_birthplaces.append(j.text)

                author_info = {
                'birthdate':author_birthdates,
                'birthplace':author_birthplaces
                }
    return author_birthdates, author_birthplaces





#Main function
all_authors3 = []
urls = get_urls(soup)
all_quotes = []
for url1 in urls:
    # print(url1)
    author_names,quote_l = quotes(url1)
  #  all_author_urls = get_author_links(url1)
    #author_birthdate, author_birthplace = get_author_birthdate()
    for author in author_names:
        all_authors3.append(author)
    # print(quote_l)
    next_page = soup.select('li.next > a')
    if not next_page or not next_page[0].get('href'):
        break
    url1 = url1 + next_page[0].get('href').lstrip('/')
    # print(url1)
    all_quotes.append(quote_l)
    #print(len(all_quotes))
new_list = []
for i in all_quotes:
    for j in i:
        new_list.append(j)
quotes_to_scrap = pd.DataFrame()
quotes_to_scrap['Quotes'] = new_list
quotes_to_scrap['Author'] = all_authors3
#quotes_to_scrap['Author_birthdate'] = author_birthdate
#quotes_to_scrap['Author_birthplace'] = author_birthplace
#print(quotes_to_scrap)

#all_author_urls = []
#for url in urls:
 #       author_urls = get_author_links(url)
  #      for j in author_urls:
   #         final_url = base_url + j.lstrip('/')
    #        all_author_urls.append(final_url)

#author_birthdates, author_birthplaces = get_author_birthdate(all_author_urls)
#quotes_to_scrap['Author_birthdate'] = author_birthdates
#quotes_to_scrap['Author_birthplace'] = author_birthplaces

print(quotes_to_scrap)












                                               Quotes               Author
0   “It is better to be hated for what you are tha...           André Gide
1   “This life is what you make it. No matter what...       Marilyn Monroe
2   “You may not be her first, her last, or her on...           Bob Marley
3   “The opposite of love is not hate, it's indiff...          Elie Wiesel
4   “It is not a lack of love, but a lack of frien...  Friedrich Nietzsche
..                                                ...                  ...
68  “The truth." Dumbledore sighed. "It is a beaut...         J.K. Rowling
69  “Never tell the truth to people who are not wo...           Mark Twain
70  “A day without sunshine is like, you know, nig...         Steve Martin
71  “Life is like riding a bicycle. To keep your b...      Albert Einstein
72                 “He's like a drug for you, Bella.”      Stephenie Meyer

[73 rows x 2 columns]


In [533]:
all_author_urls = []
for url in urls:
        author_urls = get_author_links(url)
        for j in author_urls:
            final_url = base_url + j.lstrip('/')
            all_author_urls.append(final_url)

author_birthdates, author_birthplaces = get_author_birthdate(all_author_urls)
quotes_to_scrap['Author_birthdate'] = author_birthdates
quotes_to_scrap['Author_birthplace'] = author_birthplaces

In [539]:
quotes_to_scrap

,Quotes,Author,Author_birthdate,Author_birthplace
0,“It is better to be hated for what you are tha...,André Gide,"November 22, 1869","in Paris, France"
1,“This life is what you make it. No matter what...,Marilyn Monroe,"June 01, 1926",in The United States
2,"“You may not be her first, her last, or her on...",Bob Marley,"February 06, 1945","in Nine Mile, Saint Ann, Jamaica"
3,"“The opposite of love is not hate, it's indiff...",Elie Wiesel,"September 30, 1928","in Sighet, Romania"
4,"“It is not a lack of love, but a lack of frien...",Friedrich Nietzsche,"October 15, 1844","in Röcken bei Lützen, Prussian Province of Sax..."
...,...,...,...,...
68,"“The truth."" Dumbledore sighed. ""It is a beaut...",J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U..."
69,“Never tell the truth to people who are not wo...,Mark Twain,"November 30, 1835","in Florida, Missouri, The United States"
70,"“A day without sunshine is like, you know, nig...",Steve Martin,"August 14, 1945","in Waco, Texas, The United States"
71,“Life is like riding a bicycle. To keep your b...,Albert Einstein,"March 14, 1879","in Ulm, Germany"


In [536]:
for author in data_author:
    birthplace = author.find_all('span', class_='author-born-location')

[]